In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#load data
train_df=pd.read_csv('../cleaned_data/train_data.csv')
test_df=pd.read_csv('../cleaned_data/test_data.csv')

In [4]:
y_train=train_df['Default']
X_train=train_df.drop(columns=['Default'])
y_test=test_df['Default']
X_test=test_df.drop(columns=['Default'])
X_train.shape

(163421, 31)

In [5]:
model=XGBClassifier(seed=42)
param_grid={
    'n_estimators':[80,100],
    'max_depth':[3,5,7,10],
    'learning_rate':[0.01,0.1,0.2]
}
grid_search=RandomizedSearchCV(estimator=model,param_distributions=param_grid,
                               n_iter=80,cv=8,verbose=0,
                               random_state=42,n_jobs=-1)
grid_search.fit(X_train,y_train)
best_model=grid_search.best_estimator_
y_pred=best_model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
accuracy

/home/kandarp/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 24 is smaller than n_iter=80. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/

0.8855002937145094

In [7]:
best_model.fit(X_test,y_test)
final_test=pd.read_csv('../data/test.csv')
loan_id_lists=final_test['LoanID']
final_test.drop(columns=['LoanID'],inplace=True)
#one hot encode the test data
final_test=pd.get_dummies(final_test,columns=final_test.select_dtypes(include=['object','category']).columns,dtype=int)

final_pred=best_model.predict(final_test)
#write the prediction 
with open('xgboost_predictions.csv','w') as f:
    f.write("LoanID,Default\n")
    for i in range(final_test.shape[0]):
        f.write(f'{loan_id_lists[i]},{final_pred[i]}\n')